# text preprocessing and tokenization

In [3]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

"""!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O data/sarcasm.json
"""
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


with open("data/sarcasm.json", 'r') as f:
    datastore = json.load(f)


sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# create and train a 1D ConvNet

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
conv1d (Conv1D)              (None, 116, 128)          10368     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 29,489
Trainable params: 29,489
Non-trainable params: 0
_________________________________________________________________


In [5]:
num_epochs = 50
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

Train on 20000 samples, validate on 6709 samples
Epoch 1/50
20000/20000 [==============================] - 14s 716us/sample - loss: 0.4693 - acc: 0.7666 - val_loss: 0.4232 - val_acc: 0.8097
Epoch 2/50
20000/20000 [==============================] - 14s 709us/sample - loss: 0.3621 - acc: 0.8370 - val_loss: 0.3966 - val_acc: 0.8185
Epoch 3/50
20000/20000 [==============================] - 14s 693us/sample - loss: 0.3236 - acc: 0.8554 - val_loss: 0.3874 - val_acc: 0.8201
Epoch 4/50
20000/20000 [==============================] - 13s 671us/sample - loss: 0.2905 - acc: 0.8751 - val_loss: 0.3994 - val_acc: 0.8198
Epoch 5/50
20000/20000 [==============================] - 14s 675us/sample - loss: 0.2578 - acc: 0.8931 - val_loss: 0.4256 - val_acc: 0.8150
Epoch 6/50
20000/20000 [==============================] - 14s 690us/sample - loss: 0.2255 - acc: 0.9068 - val_loss: 0.4577 - val_acc: 0.8152
Epoch 7/50
20000/20000 [==============================] - 14s 692us/sample - loss: 0.1938 - acc: 0.9225 -

KeyboardInterrupt: 

# visualize the training and validation accuracy:

In [6]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'acc')
plot_graphs(history, 'loss')

NameError: name 'history' is not defined